[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AndreVuillemot160/TerrColab/blob/main/TerrColab.ipynb)

```
___________                  _________        .__        ___.    
\__    ___/_________________ \_   ___ \  ____ |  | _____ \_ |__  
  |    |_/ __ \_  __ \_  __ \/    \  \/ /  _ \|  | \__  \ | __ \ 
  |    |\  ___/|  | \/|  | \/\     \___(  <_> )  |__/ __ \| \_\ \
  |____| \___  >__|   |__|    \______  /\____/|____(____  /___  /
             \/                      \/                 \/    \/ 
```
**Run a Terraria Server on Google Colab!**

---

The script below will run your server. You'll have to create a server with `Make a Server` first to be able to use it - don't worry, the scripts below will do the majority of the work for you. You might also want to change the default region to your region, check below.

When you see `/content/drive/Shareddrives/terratia` replace it with yours foldername or make `terratia` as your exact one, me personally recommends using team drives for that because they have unlimited storage. 

In [ ]:
!pip -q install pyngrok

import os
import re
from pyngrok import conf, ngrok
# Update the package lists
!sudo apt update &>/dev/null && echo "apt cache successfully updated" || echo "apt cache update failed, you might receive stale packages"

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Ask for the ngrok authtoken
print("Get your authtoken from https://dashboard.ngrok.com/auth")
import getpass
authtoken = getpass.getpass()  # input your Ngrok auth token everytime you run the cell or simply replace "getpass.getpass()" with your token in "double quotes"
! ngrok authtoken $authtoken # login to ngrok

# Sets default ngrok region
conf.get_default().region = 'us'  # Change this to whichever region you want

# Connect to ngrok
url = ngrok.connect(7777, 'tcp')
print('Your server address is ' + ((str(url).split('"')[1::2])[0]).replace('tcp://', ''))

# Change directory to the Terraria server folder on Google Drive
%cd "/content/drive/Shareddrives/terratia" 
!ls #list the directory contents (to verify that working directory was changed)

print('Starting server...')

!chmod u+x TerrariaServer*
!./TerrariaServer -config serverconfig.txt

**Make a Terraria server**

After downloading a server, please edit the server file. By default it will save the world in the Ubuntu VM self, and select the way instead to use gdrive. 

In [ ]:
from google.colab import drive
import requests

drive.mount('/content/drive')

#Get unzip to unpack the .zip folder
!sudo apt install --yes unzip
# Create the directory which will be used for the server
! mkdir "/content/drive/Shareddrives/terratia"

print('Downloading to Google Drive...')

url = 'https://terraria.org/system/dedicated_servers/archives/000/000/046/original/terraria-server-1423.zip?1621172068'

r = requests.get(url)

if r.status_code is 200:
  with open('/content/drive/Shareddrives/terratia/terraria-server-1423.zip', 'wb') as f:
    f.write(r.content)
  print('Done!')
else:
  print('Error '+ str(r.status_code) + 'Unfortunately you was unable to get the Terraria server :/.')

#change directory to terraria server and unpack it
%cd "/content/drive/Shareddrives/terratia" 
!ls # check if we are here in our terraria gdrive
!unzip terraria-server-1423.zip
!\cp -rf /content/drive/Shareddrives/terratia/1423/Linux  /content/drive/Shareddrives/terratia

# Debug
SSH acess to host OS

In [ ]:
## For SSH acess to host OS ##
#Download ngrok
! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip
#Setup sshd
! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null
#Set root password
! echo root:terr | chpasswd #change the password (default is terr)
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
! echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
! echo "export LD_LIBRARY_PATH" >> /root/.bashrc

#Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')

#Ask token
print("Copy authtoken from https://dashboard.ngrok.com/auth")
import getpass
authtoken = getpass.getpass() #input your Ngrok auth token everytime or simply replace "getpass.getpass()" with your token in "double quotes"

#Create tunnel
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')

#Get the public address by running the cell below (for some reason, running both the commands in same cell result in an error. 
#Thus it has been kept in a sepreate cell)

In [2]:
#Get public address 
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [ ]:
## For inspecting the terraria server directory ##
%cd "/content/drive/Shareddrives/terratia" #change directory here as well
!ls